# Install pyodbc

https://docs.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver15#installing-microsoft-odbc-driver-for-sql-server

https://www.sqlshack.com/performing-crud-operations-with-a-python-sql-library-for-sql-server/

https://stackoverflow.com/questions/33725862/connecting-to-microsoft-sql-server-using-python/33787509

In [1]:
import pyodbc
server = 'officelaptop'
database = 'DBA'
username = 'grafana' 
password = 'grafana' 
app_name = 'pyodbc-test.py'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+'; TrustServerCertificate=yes; APP={app_name};')
cursor = cnxn.cursor()

In [ ]:
# Sample Query
cursor.execute("select param_key, param_value from dbo.sma_params") 
row = cursor.fetchone() 
while row: 
    print(row)
    row = cursor.fetchone()

In [19]:
sum = lambda x,y: x+y

print(sum(10,5))

15


In [2]:
# Sample Query
cursor.execute("select top 3 param_key, param_value from dbo.sma_params") 
query_resultset = cursor.fetchall() 
col_names = [column[0] for column in query_resultset[0].cursor_description]
#print(col_names)

#col_index = lambda col: col_names.index(col)
#col_value = lambda row, col: row[col_index(col)]
col_value = lambda row, col: row[(lambda col: col_names.index(col))(col)]

for row in query_resultset:
    param_key = col_value(row,'param_key')
    param_value = col_value(row,'param_value')
    print(f"{param_key} => {param_value}")

alert_sender_email => sqlagentservice@gmail.com
credential_manager_database => DBA
cto_email_id => sqlagentservice@gmail.com


In [39]:
# Get Data, convert data unit, and print pretty table
cursor.execute("select top 5 * from tempdb_space_usage_all_servers") 
pyodbc_query_resultset = cursor.fetchall() 
col_names = [column[0] for column in query_resultset[0].cursor_description]
print(col_names)


['sql_instance', 'data_size_mb', 'data_used_mb', 'data_used_pct', 'log_size_mb', 'log_used_mb', 'log_used_pct', 'version_store_mb', 'version_store_pct', 'updated_date_utc', 'collection_time_utc']


In [45]:
def get_pretty_data_size(size:float,unit:str='mb',precision:int=2):
    unit = unit.lower()
    suffixes=['b', 'kb','mb','gb','tb']
    suffixIndex = suffixes.index(unit)
    while size > 1024 and suffixIndex < (len(suffixes)-1):
        suffixIndex += 1 #increment the index of the suffix
        size = size/1024.0 #apply the division
    return "%.*f %s"%(precision,size,suffixes[suffixIndex])

In [30]:
from prettytable import PrettyTable


#def get_pretty_table(pyodbc_query_resultset, return_pretty_data:bool=False):
if len(pyodbc_query_resultset) == 0:
    raise Exception(f"No data in passed resulset.")
else:
    pt_results = PrettyTable()
    col_names = [column[0] for column in pyodbc_query_resultset[0].cursor_description]
    pt_results.field_names = col_names
    pt_results.add_rows(pyodbc_query_resultset)

    suffixes=['kb','mb','gb','tb']
    for col in col_names:
        for unit in suffixes:
            if col.endswith(f"_{unit}"):
                print(col)
                pt_results.custom_format = {
                    col: lambda field, value: f"{value:,} mb"
                }

print(get_pretty_data_size(6560,'mb'))

#custom_format_dict = dict(data_size_mb = lambda field, value: f"{value:,}")
custom_format_dict = dict(data_size_mb = lambda field, value: get_pretty_data_size(float(value),'mb'))

pt_results.custom_format = custom_format_dict

pt_results.custom_format["version_store_mb"] = lambda field, value: get_pretty_data_size(float(value),'mb')

    #return pt_results
print(pt_results.get_string(fields=["sql_instance","data_size_mb","log_used_pct","version_store_mb"]))

data_size_mb
data_used_mb
log_size_mb
log_used_mb
version_store_mb
6.41 gb
+-----------------+--------------+--------------+------------------+
|   sql_instance  | data_size_mb | log_used_pct | version_store_mb |
+-----------------+--------------+--------------+------------------+
| 21L-LTPABL-1187 |  800.00 mb   |    25.26     |     28.00 mb     |
|    SQLMonitor   |   6.41 gb    |     6.88     |     0.44 mb      |
+-----------------+--------------+--------------+------------------+


In [66]:
from prettytable import PrettyTable

return_pretty_data = True
#def get_pretty_table(pyodbc_query_resultset, return_pretty_data:bool=False):
if len(pyodbc_query_resultset) == 0:
    raise Exception(f"No data in passed resulset.")
else:
    pt_results = PrettyTable()
    col_names = [column[0] for column in pyodbc_query_resultset[0].cursor_description]
    pt_results.field_names = col_names
    pt_results.add_rows(pyodbc_query_resultset)

    print('Before custom format')
    print(pt_results.get_string(fields=["sql_instance","data_size_mb","log_used_pct","version_store_mb"]))

    if return_pretty_data:
        suffixes=['kb','mb','gb','tb']
        init = True
        for col in col_names:
            for unit in suffixes:
                if col.endswith(f"_{unit}"):
                    print(f"col={col}, unit={unit}")
                    if init:
                        pt_results.custom_format = { col: lambda field, value: get_pretty_data_size(float(value),field[-2:]) }
                        init = False
                    else:
                        pt_results.custom_format[col] = lambda field, value: get_pretty_data_size(float(value),field[-2:])

print(get_pretty_data_size(float(6560.00),'mb'))
#custom_format_dict = dict(data_size_mb = lambda field, value: f"{value:,}")
#custom_format_dict = dict()
#custom_format_dict["data_size_mb"] = lambda field, value: get_pretty_data_size(float(value),'mb')
#custom_format_dict["version_store_mb"] = lambda field, value: get_pretty_data_size(float(value),'mb')

#pt_results.custom_format["version_store_mb"] = lambda field, value: get_pretty_data_size(float(value),'mb')

#name = 'Ajay Dwivedi'
#print(name[-3:])

    #return pt_results
print('After custom format')
print(pt_results.get_string(fields=["sql_instance","data_size_mb","log_used_pct","version_store_mb"]))

Before custom format
+-----------------+--------------+--------------+------------------+
|   sql_instance  | data_size_mb | log_used_pct | version_store_mb |
+-----------------+--------------+--------------+------------------+
| 21L-LTPABL-1187 |    800.00    |    34.58     |      48.44       |
|    SQLMonitor   |   6560.00    |     6.44     |       0.25       |
+-----------------+--------------+--------------+------------------+
col=data_size_mb, unit=mb
col=data_used_mb, unit=mb
col=log_size_mb, unit=mb
col=log_used_mb, unit=mb
col=version_store_mb, unit=mb
6.41 gb
After custom format
+-----------------+--------------+--------------+------------------+
|   sql_instance  | data_size_mb | log_used_pct | version_store_mb |
+-----------------+--------------+--------------+------------------+
| 21L-LTPABL-1187 |  800.00 mb   |    34.58     |     48.44 mb     |
|    SQLMonitor   |   6.41 gb    |     6.44     |     0.25 mb      |
+-----------------+--------------+--------------+-----------

In [ ]:
# Sample Query

cursor.execute("select param_key, param_value from dbo.sma_params") 
result = cursor.fetchall() 
for row in result:
    #print(row)
    pass

In [ ]:
from prettytable import PrettyTable

pt_results = PrettyTable()
pt_results.field_names = [column[0] for column in result[0].cursor_description]
pt_results.add_rows(result)

print(pt_results)


In [ ]:
import pandas as pd
from pandas import DataFrame

df_columns = [column[0] for column in result[0].cursor_description]
df_results = pd.DataFrame.from_records(result,columns=df_columns)

print(df_results)

In [ ]:
#print(result)
#[row.param_key for row in result]

index_col = 'param_key'
columns = [column[0] for column in result[0].cursor_description]
indexOfIndexCol = columns.index(index_col)
#print(indexOfIndexCol)

#result[indexOfIndexCol]
indexValues = [row[indexOfIndexCol] for row in result]
print(indexValues)

for row in result:
    #print(row[indexOfIndexCol])
    pass


#[row.param_key for row in result]


**Connection String with APP name**

In [ ]:
import pyodbc
cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost;'
                      'Database=StackOverflow;'
                      'Trusted_Connection=yes;'
                      'APP=sql-server-load-generator.py')
cursor = cnxn.cursor()

# Sample Query
cursor.execute("SELECT @@version; WAITFOR DELAY '00:20:00'") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()